In [2]:
import xarray as xr
import numpy as np
import pandas as pd

In [3]:
ERA5 = xr.open_dataset('/home/m/m301036/josie/LSAT/1900-2022LSAT/SpatialPattern/SLP/slp_ERA5_ano.nc')
ERA5_ano =ERA5.sel(time=slice('2008-01-01','2022-12-31'))

ERA5_ano


<xarray.Dataset>
Dimensions:    (time: 180, longitude: 360, latitude: 91)
Coordinates:
  * time       (time) datetime64[ns] 2008-01-01 2008-02-01 ... 2022-12-01
  * longitude  (longitude) float32 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * latitude   (latitude) float32 0.0 1.0 2.0 3.0 4.0 ... 87.0 88.0 89.0 90.0
    month      (time) int64 ...
Data variables:
    msl        (time, latitude, longitude) float64 ...

In [4]:
ERA5_ano['DJF'] = ERA5_ano['msl'].sel(time=ERA5_ano['time.season']=='DJF').groupby('time.year').mean('time')
ERA5_ano['MAM'] = ERA5_ano['msl'].sel(time=ERA5_ano['time.season']=='MAM').groupby('time.year').mean('time')
ERA5_ano['JJA'] = ERA5_ano['msl'].sel(time=ERA5_ano['time.season']=='JJA').groupby('time.year').mean('time')
ERA5_ano['SON'] = ERA5_ano['msl'].sel(time=ERA5_ano['time.season']=='SON').groupby('time.year').mean('time')

In [5]:
ERA5_ano['DJF']

<xarray.DataArray 'DJF' (year: 15, latitude: 91, longitude: 360)>
array([[[ -61.0421748 ,  -58.74983062,  -57.44954692, ...,
          -64.03914719,  -62.91207232,  -61.67221375],
        [ -57.4789126 ,  -56.02011348,  -54.6639143 , ...,
          -61.85871867,  -60.33193598,  -59.18519224],
        [ -53.39695545,  -52.602854  ,  -50.54776423, ...,
          -59.24976711,  -57.08060213,  -55.92831131],
        ...,
        [-491.31171663, -491.024729  , -490.85122375, ...,
         -492.3096206 , -491.99468581, -491.63154217],
        [-484.98750847, -484.82416582, -484.71868648, ...,
         -485.42594004, -485.30278201, -485.11504912],
        [-508.24847561, -508.24847561, -508.24847561, ...,
         -508.24847561, -508.24847561, -508.24847561]],

       [[ -47.26352896,  -46.60920562,  -46.94694275, ...,
          -45.15893886,  -45.52144732,  -46.10710959],
        [ -44.1820376 ,  -44.60344682,  -45.03370596, ...,
          -42.6373645 ,  -43.03766514,  -43.91175474],
        [ -40.63133045,  -41.81118733,  -41.82901423, ...,
          -39.68987127,  -39.92956047,  -40.94654048],
...
        [ 533.2091167 ,  536.33725017,  539.44825542, ...,
          523.78152524,  526.94021003,  530.14449949],
        [ 549.4135332 ,  550.97010501,  552.52089685, ...,
          544.68603913,  546.30138465,  547.88234671],
        [ 535.92860772,  535.92860772,  535.92860772, ...,
          535.92860772,  535.92860772,  535.92860772]],

       [[ -18.21925813,  -17.27326812,  -17.27246358, ...,
          -15.57820969,  -16.18811399,  -16.91960959],
        [ -16.9632876 ,  -16.75969682,  -16.70818513, ...,
          -15.613927  ,  -15.86839431,  -16.69821308],
        [ -15.09747629,  -15.555979  ,  -14.75609756, ...,
          -15.12476711,  -14.97643547,  -15.70174881],
        ...,
        [-278.87421663, -280.66014566, -282.60643208, ...,
         -273.85128726, -275.50770664, -277.16800051],
        [-234.7817793 , -235.68093665, -236.68483232, ...,
         -232.28010671, -233.12049035, -233.94577829],
        [-220.46722561, -220.46722561, -220.46722561, ...,
         -220.46722561, -220.46722561, -220.46722561]]])
Coordinates:
  * longitude  (longitude) float32 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * latitude   (latitude) float32 0.0 1.0 2.0 3.0 4.0 ... 87.0 88.0 89.0 90.0
  * year       (year) int64 2008 2009 2010 2011 2012 ... 2019 2020 2021 2022

## Define function

In [6]:
from scipy.stats import linregress

def calc_trend(data):
    slope, intercept, r_value, p_value, std_err = linregress(np.arange(len(data)), data)  
    return slope, p_value

## ERA5 seasonal trend pattern

In [7]:
ERA5_ano['DJF_trend'], ERA5_ano['p_value_DJF'] = xr.apply_ufunc(calc_trend, ERA5_ano['DJF'], input_core_dims=[['year']], output_core_dims=[[], []], vectorize=True)
ERA5_ano['MAM_trend'], ERA5_ano['p_value_MAM'] = xr.apply_ufunc(calc_trend, ERA5_ano['MAM'], input_core_dims=[['year']], output_core_dims=[[], []], vectorize=True)
ERA5_ano['JJA_trend'], ERA5_ano['p_value_JJA'] = xr.apply_ufunc(calc_trend, ERA5_ano['JJA'], input_core_dims=[['year']], output_core_dims=[[], []], vectorize=True)
ERA5_ano['SON_trend'], ERA5_ano['p_value_SON'] = xr.apply_ufunc(calc_trend, ERA5_ano['SON'], input_core_dims=[['year']], output_core_dims=[[], []], vectorize=True)

In [8]:
slp_trend = xr.Dataset({
    'DJF': ERA5_ano['DJF_trend']*15.0,
    'MAM': ERA5_ano['MAM_trend']*15.0,
    'JJA': ERA5_ano['JJA_trend']*15.0,
    'SON': ERA5_ano['SON_trend']*15.0
}, coords={'longitude': ERA5_ano['longitude'], 'latitude': ERA5_ano['latitude'], 'season': ['DJF', 'MAM', 'JJA', 'SON']})
slp_trend

<xarray.Dataset>
Dimensions:    (longitude: 360, latitude: 91, season: 4)
Coordinates:
  * longitude  (longitude) float32 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * latitude   (latitude) float32 0.0 1.0 2.0 3.0 4.0 ... 87.0 88.0 89.0 90.0
  * season     (season) <U3 'DJF' 'MAM' 'JJA' 'SON'
Data variables:
    DJF        (latitude, longitude) float64 64.1 63.76 63.44 ... 0.2486 0.2486
    MAM        (latitude, longitude) float64 100.0 98.92 97.76 ... -346.9 -346.9
    JJA        (latitude, longitude) float64 27.17 26.52 25.85 ... -429.6 -429.6
    SON        (latitude, longitude) float64 10.67 11.71 12.72 ... 198.8 198.8

In [36]:
slp_trend['DJF'].min().values

array(-978.42382812)

In [9]:
sig_ds = xr.Dataset({
    'DJF': ERA5_ano['p_value_DJF'],
    'MAM': ERA5_ano['p_value_MAM'],
    'JJA': ERA5_ano['p_value_JJA'],
    'SON': ERA5_ano['p_value_SON']
}, coords={'longitude': ERA5_ano['longitude'], 'latitude': ERA5_ano['latitude'], 'season': ['DJF', 'MAM', 'JJA', 'SON']})
sig_ds

<xarray.Dataset>
Dimensions:    (longitude: 360, latitude: 91, season: 4)
Coordinates:
  * longitude  (longitude) float32 0.0 1.0 2.0 3.0 ... 356.0 357.0 358.0 359.0
  * latitude   (latitude) float32 0.0 1.0 2.0 3.0 4.0 ... 87.0 88.0 89.0 90.0
  * season     (season) <U3 'DJF' 'MAM' 'JJA' 'SON'
Data variables:
    DJF        (latitude, longitude) float64 0.1352 0.1346 ... 0.9995 0.9995
    MAM        (latitude, longitude) float64 0.02583 0.02662 ... 0.4179 0.4179
    JJA        (latitude, longitude) float64 0.4917 0.4995 ... 0.314 0.314
    SON        (latitude, longitude) float64 0.7848 0.7599 ... 0.5391 0.5391

## MPI-ESM-Trend Calculating

In [10]:
data_hist_ssp245_MPI_ESM  = '/work/mh0033/m301036/LSAT/CMIP6-MPI-M-LR/MergeDataOut/psl_Amon_1850-2022_*.nc'
ds = xr.open_mfdataset(data_hist_ssp245_MPI_ESM, combine = 'nested', concat_dim = 'run')
ds

<xarray.Dataset>
Dimensions:    (time: 2076, run: 30, bnds: 2, lon: 180, lat: 90)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lon        (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * lat        (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
Dimensions without coordinates: run, bnds
Data variables:
    time_bnds  (run, time, bnds) datetime64[ns] dask.array<chunksize=(1, 2076, 2), meta=np.ndarray>
    psl        (run, time, lat, lon) float32 dask.array<chunksize=(1, 2076, 90, 180), meta=np.ndarray>
Attributes: (12/49)
    CDI:                    Climate Data Interface version 2.0.3 (https://mpi...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    institution:            Max Planck Institute for Meteorology, Hamburg 201...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variable_id:            psl
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/2d4e90e1-d70a-49f4-8374-eb769a0b143d
    CDO:                    Climate Data Operators version 2.0.3 (https://mpi...

In [11]:
# #Perfrom land sea mask
# land_sea_mask=xr.open_dataset('/work/mh0033/m301036/LSAT/CMIP6-MPI-M-LR/GR15_lsm_regrid.nc')
# # land_sea_mask.coords
# display(land_sea_mask)
# # land_sea_mask.coords

In [12]:
# mask_data = land_sea_mask['var1']
# mask_data
# # Align the time coordinates between the mask dataset and the original dataset
# time_values = pd.to_datetime(mask_data['time'].values, format='mixed', dayfirst=True)
# mask_data['time'] = time_values

# # Align the time coordinates between the mask dataset and the original dataset
# mask_data = mask_data.reindex(time=ds['time'], method='nearest')

# # Apply the land-sea mask to the original dataset
# masked_tas = ds.where(mask_data == 1, drop=False)
# masked_tas

In [13]:
slp = ds['psl'].loc[:,'2008-01-01':'2022-12-31',0:90,:]
# slp = slp - 273.15
print(slp.min().values)
slp_climatology = slp.groupby('time.month').mean(dim='time')
slp_climatology
slp_ano = slp.groupby('time.month') - slp_climatology
slp_ano_ds = slp_ano.to_dataset()
slp_ano_ds

97214.24


/home/m/m301036/.conda/envs/mykernel/lib/python3.9/site-packages/xarray/core/indexing.py:1440: PerformanceWarning: Slicing with an out-of-order index is generating 15 times more chunks
  return self.array[key]


<xarray.Dataset>
Dimensions:  (time: 180, lon: 180, lat: 45, run: 30)
Coordinates:
  * time     (time) datetime64[ns] 2008-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: run
Data variables:
    psl      (run, time, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>

In [14]:
seasons = ['JJA', 'DJF', 'MAM', 'SON']
season_means = {}

for season in seasons:
    if season == 'JJA':
        months = [6,7,8]
    elif season == 'DJF':
        months =[12,1,2]
    elif season == 'MAM':
        months = [3,4,5]
    elif season == 'SON':
        months = [9,10,11]

    season_months = slp_ano_ds.sel(time=slp_ano_ds.time.dt.month.isin(months))
    
    # Calculate the seasonal mean SAT anomalies
    season_mean_anomalies = season_months.groupby('time.year').mean('time')
    
    # Store the seasonal mean in the dictionary
    season_means[season] = season_mean_anomalies['psl']

# Access the multiyear JJA mean SAT anomalies
    
slp_ano_ds['JJA'] = season_means['JJA']
slp_ano_ds['DJF'] = season_means['DJF']
slp_ano_ds['MAM'] = season_means['MAM']
slp_ano_ds['SON'] = season_means['SON']

slp_ano_ds

<xarray.Dataset>
Dimensions:  (time: 180, lon: 180, lat: 45, run: 30, year: 15)
Coordinates:
  * time     (time) datetime64[ns] 2008-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
  * year     (year) int64 2008 2009 2010 2011 2012 ... 2018 2019 2020 2021 2022
Dimensions without coordinates: run
Data variables:
    psl      (run, time, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>
    JJA      (year, run, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>
    DJF      (year, run, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>
    MAM      (year, run, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>
    SON      (year, run, lat, lon) float32 dask.array<chunksize=(1, 1, 45, 180), meta=np.ndarray>

In [15]:
import scipy.stats as stats
from scipy.stats import linregress

def calc_trend(data):
    slope, intercept, r_value, p_value, std_err = linregress(np.arange(len(data)), data)  
    return slope, p_value

In [16]:
slp_ano_ds['slope_JJA'], slp_ano_ds['p_value_JJA'] = xr.apply_ufunc(calc_trend, slp_ano_ds['JJA'].chunk(dict(run=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
slp_ano_ds['slope_JJA'].attrs['units'] = 'Pa/15yrs'
slp_ano_ds['p_value_JJA'].attrs['units'] = 'p_value'

In [17]:
slp_ano_ds['slope_DJF'], slp_ano_ds['p_value_DJF'] = xr.apply_ufunc(calc_trend, slp_ano_ds['DJF'].chunk(dict(run=-1, year=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
slp_ano_ds['slope_DJF'].attrs['units'] = 'Pa/15yrs'
slp_ano_ds['p_value_DJF'].attrs['units'] = 'p_value'

In [18]:
slp_ano_ds['slope_MAM'], slp_ano_ds['p_value_MAM'] = xr.apply_ufunc(calc_trend, slp_ano_ds['MAM'].chunk(dict(run=-1, year=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
slp_ano_ds['slope_MAM'].attrs['units'] = '°C/15yrs'
slp_ano_ds['p_value_MAM'].attrs['units'] = 'p_value'

In [19]:
slp_ano_ds['slope_SON'], slp_ano_ds['p_value_SON'] = xr.apply_ufunc(calc_trend, slp_ano_ds['SON'].chunk(dict(run=-1, year=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
slp_ano_ds['slope_SON'].attrs['units'] = '°C/15yrs'
slp_ano_ds['p_value_SON'].attrs['units'] = 'p_value'

In [20]:
slp_ano_ds = slp_ano_ds.compute()

In [21]:
slp_ano_ds

<xarray.Dataset>
Dimensions:      (time: 180, lon: 180, lat: 45, run: 30, year: 15)
Coordinates:
  * time         (time) datetime64[ns] 2008-01-16T12:00:00 ... 2022-12-16T12:...
  * lon          (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * lat          (lat) float64 1.0 3.0 5.0 7.0 9.0 ... 81.0 83.0 85.0 87.0 89.0
    month        (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 3 4 5 6 7 8 9 10 11 12
  * year         (year) int64 2008 2009 2010 2011 2012 ... 2019 2020 2021 2022
Dimensions without coordinates: run
Data variables: (12/13)
    psl          (run, time, lat, lon) float32 114.2 113.5 ... -124.0 -124.7
    JJA          (year, run, lat, lon) float32 9.607 9.172 ... -229.8 -228.5
    DJF          (year, run, lat, lon) float32 80.62 79.16 79.93 ... 517.5 516.2
    MAM          (year, run, lat, lon) float32 22.95 20.33 ... -667.7 -669.8
    SON          (year, run, lat, lon) float32 6.216 7.505 8.391 ... 222.4 224.1
    slope_JJA    (run, lat, lon) float64 1.913 2.013 2.18 ... 21.24 21.29 21.34
    ...           ...
    slope_DJF    (run, lat, lon) float64 -3.338 -3.314 -3.369 ... -4.773 -4.985
    p_value_DJF  (run, lat, lon) float64 0.3422 0.3345 0.3248 ... 0.88 0.8748
    slope_MAM    (run, lat, lon) float64 0.74 0.8462 0.7824 ... -0.5738 -0.7293
    p_value_MAM  (run, lat, lon) float64 0.7228 0.6758 0.7048 ... 0.9788 0.9732
    slope_SON    (run, lat, lon) float64 0.07541 0.0251 0.1772 ... -13.93 -13.9
    p_value_SON  (run, lat, lon) float64 0.9744 0.9912 0.9377 ... 0.4857 0.487

## Selection of the minimum and maximum five models

In [22]:
model_15yr_JJA = np.genfromtxt(fname='/home/m/m301036/josie/LSAT/1900-2022LSAT/MPI-ESM-LR_NH_SATAs_15yr_JJA_trend.txt',delimiter='\t', skip_header=1)
model_15yr_DJF = np.genfromtxt(fname='/home/m/m301036/josie/LSAT/1900-2022LSAT/MPI-ESM-LR_NH_SATAs_15yr_DJF_trend.txt',delimiter='\t', skip_header=1)

DJF_15yr = xr.DataArray(model_15yr_DJF[:,1], dims=['run'], coords={'run': np.arange(1, 31, 1)})
JJA_15yr = xr.DataArray(model_15yr_JJA[:,1], dims=['run'], coords={'run': np.arange(1, 31, 1)})

# assume you have an xarray called 'xr_data'
# get the most minimum five values and their indices
min_indices = np.argpartition(DJF_15yr.values.flatten(), 5)[:5]
min_values = DJF_15yr.values.flatten()[min_indices]
min_coords = np.unravel_index(min_indices, DJF_15yr.shape)

# create a new xarray to store the most minimum values
min_xr = xr.DataArray(min_values, dims=['value'], coords={'value': np.arange(5)})

# print the new xarray
print(min_xr)
print(min_coords)

<xarray.DataArray (value: 5)>
array([-0.48796285, -0.37097916, -0.23312506, -0.10872448, -0.00871999])
Coordinates:
  * value    (value) int64 0 1 2 3 4
(array([20,  6, 11, 16, 17]),)


In [23]:
max_indices = np.argpartition(DJF_15yr.values.flatten(), -5)[-5:]
max_values = DJF_15yr.values.flatten()[max_indices]
max_coords = np.unravel_index(max_indices, DJF_15yr.shape)

max_xr = xr.DataArray(max_values, dims=['value'], coords={'value': np.arange(5)})
print(max_xr)
print(max_coords)
type(max_coords)
max_coords

<xarray.DataArray (value: 5)>
array([0.96829494, 1.00914491, 1.20347835, 1.35353672, 1.45603341])
Coordinates:
  * value    (value) int64 0 1 2 3 4
(array([15, 21, 28,  1, 10]),)


(array([15, 21, 28,  1, 10]),)

In [24]:
max_JJA_indices = np.argpartition(JJA_15yr.values.flatten(), -5)[-5:]
max_JJA_values = JJA_15yr.values.flatten()[max_JJA_indices]
max_JJA_coords = np.unravel_index(max_JJA_indices, JJA_15yr.shape)

max_JJA_xr = xr.DataArray(max_JJA_values, dims=['value'], coords={'value': np.arange(5)})
print(max_JJA_xr)
print(max_JJA_coords)
type(max_JJA_coords)
max_JJA_coords

<xarray.DataArray (value: 5)>
array([0.65486606, 0.68288031, 0.71153335, 0.83900355, 0.83478993])
Coordinates:
  * value    (value) int64 0 1 2 3 4
(array([15,  2, 21, 28, 16]),)


(array([15,  2, 21, 28, 16]),)

In [25]:
min_JJA_indices = np.argpartition(JJA_15yr.values.flatten(), 5)[:5]
min_JJA_values = JJA_15yr.values.flatten()[min_JJA_indices]
min_JJA_coords = np.unravel_index(min_JJA_indices, JJA_15yr.shape)

min_JJA_xr = xr.DataArray(min_JJA_values, dims=['value'], coords={'value': np.arange(5)})
print(min_JJA_xr)
print(min_JJA_coords)
type(min_JJA_coords)
min_JJA_coords

<xarray.DataArray (value: 5)>
array([-0.05299711, -0.04975601, -0.00816475,  0.12322683,  0.15961614])
Coordinates:
  * value    (value) int64 0 1 2 3 4
(array([25,  8,  0,  5, 13]),)


(array([25,  8,  0,  5, 13]),)

## Plot the trend pattern of the DJF 15-yr Spatial pattern

In [26]:
slope_JJA = slp_ano_ds['slope_JJA'] 
slope_DJF = slp_ano_ds['slope_DJF']
slope_MAM = slp_ano_ds['slope_MAM']
slope_SON = slp_ano_ds['slope_SON']

p_value_JJA = slp_ano_ds['p_value_JJA']
p_value_DJF = slp_ano_ds['p_value_DJF']
p_value_MAM = slp_ano_ds['p_value_MAM']
p_value_SON = slp_ano_ds['p_value_SON']

In [27]:
slope_JJA_data = slope_JJA*15.0
slope_DJF_data = slope_DJF*15.0
slope_MAM_data = slope_MAM*15.0
slope_SON_data = slope_SON*15.0

In [28]:
slope_DJF_data

<xarray.DataArray 'slope_DJF' (run: 30, lat: 45, lon: 180)>
array([[[ -50.06542959,  -49.70354421,  -50.52929572, ...,
          -56.63853325,  -56.48046848,  -54.24818562],
        [ -47.16531888,  -43.30287465,  -42.21121819, ...,
          -49.40680713,  -51.06612677,  -51.84012166],
        [ -50.13462483,  -41.58272706,  -36.14704235, ...,
          -43.76548699,  -48.14104358,  -53.72740117],
        ...,
        [  49.66141963,   47.6876374 ,   45.78963273, ...,
           55.61105965,   53.67341016,   51.71748667],
        [  44.62401705,   43.55900017,   42.5128519 , ...,
           47.64787162,   46.66239014,   45.6050605 ],
        [  35.5983509 ,   35.20912375,   34.78572076, ...,
           37.04744313,   36.68988153,   36.34501311]],

       [[   8.56292174,    6.93777811,    4.34235345, ...,
            6.28236478,    6.05901371,    6.15373955],
        [   5.49874298,    3.87137307,    2.40499395, ...,
            2.5962622 ,    2.21065848,    2.73325924],
        [   8.4217327 ,    7.2758082 ,    6.92689725, ...,
            1.60044679,    3.67229191,    6.39550837],
...
        [-718.88978781, -718.54506724, -717.73465061, ...,
         -717.05830847, -718.0472799 , -718.6963095 ],
        [-663.3016305 , -661.53026881, -659.61802074, ...,
         -667.76380512, -666.38935723, -664.90766001],
        [-605.84346444, -604.51241112, -603.1276474 , ...,
         -610.69768729, -609.5495224 , -608.35169056]],

       [[  55.29938497,   55.29296897,   58.10072488, ...,
           56.61062987,   55.46623882,   54.83286947],
        [  61.55636328,   62.14271818,   67.02957712, ...,
           58.99916347,   59.42229417,   61.24093452],
        [  67.97474957,   70.97753924,   77.77817682, ...,
           59.96107522,   61.87304558,   66.45507952],
        ...,
        [-141.13030543, -152.68205643, -164.54981286, ...,
         -108.55160169, -119.04505947, -129.9355466 ],
        [-112.57352645, -119.63141952, -126.69001171, ...,
          -91.74387748,  -98.61077397, -105.5855247 ],
        [ -80.98465184,  -84.05553327,  -87.09682437, ...,
          -68.39412464,  -71.59975822,  -74.77818407]]])
Coordinates:
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
Dimensions without coordinates: run

In [29]:
slope_JJA_MME = slope_JJA_data.mean(dim='run')
slope_DJF_MME = slope_DJF_data.mean(dim='run')
slope_MAM_MME = slope_MAM_data.mean(dim='run')
slope_SON_MME = slope_SON_data.mean(dim='run')

In [30]:
#using the for loop to pick up the min5 and max5 data of DJF 


# Extract trend spatial data for minimum five runs in DJF
DJF_min5_trend = []
for i in range(5):
    run_index = min_coords[0][i]
    DJF_min5_trend.append(slope_DJF_data[run_index,:,:])
DJF_min5_trend = xr.concat(DJF_min5_trend, dim='run')

# Extract trend spatial data for maximum five runs in DJF
DJF_max5_trend = []
for i in range(5):
    run_index = max_coords[0][i]
    DJF_max5_trend.append(slope_DJF_data[run_index,:,:])
DJF_max5_trend = xr.concat(DJF_max5_trend, dim='run')

# Extract trend spatial data for minimum five runs in JJA
JJA_min5_trend = []
for i in range(5):
    run_index = min_JJA_coords[0][i]
    JJA_min5_trend.append(slope_JJA_data[run_index,:,:])
JJA_min5_trend = xr.concat(JJA_min5_trend, dim='run')

# Extract trend spatial data for maximum five runs in JJA
JJA_max5_trend = []
for i in range(5):
    run_index = max_JJA_coords[0][i]
    JJA_max5_trend.append(slope_JJA_data[run_index,:,:])
JJA_max5_trend = xr.concat(JJA_max5_trend, dim='run')


In [31]:
JJA_max5_trend

<xarray.DataArray 'slope_JJA' (run: 5, lat: 45, lon: 180)>
array([[[ -20.17089805,  -21.23897961,  -22.8430532 , ...,
          -25.06487261,  -23.58398501,  -21.68931359],
        [ -21.03236579,  -20.77553021,  -21.0099056 , ...,
          -25.16894455,  -23.70131256,  -22.09096065],
        [ -18.60867806,  -17.66364369,  -18.06961449, ...,
          -22.32896231,  -19.96023991,  -18.75613864],
        ...,
        [ -79.96595376,  -77.93834475,  -75.8931402 , ...,
          -86.28584954,  -84.14899656,  -82.02608779],
        [-104.62961483, -103.47796195, -102.31083267, ...,
         -108.15484878, -106.96163024, -105.77134231],
        [-110.608956  , -110.02372006, -109.40918735, ...,
         -112.92689773, -112.35184172, -111.77971254]],

       [[ -68.53613305,  -66.53194652,  -64.57924094, ...,
          -65.5484096 ,  -66.1014224 ,  -67.97670257],
        [ -67.01422794,  -65.77050877,  -63.2011717 , ...,
          -65.87220934,  -65.89299682,  -66.25795236],
        [ -69.06082555,  -68.86077149,  -65.98688558, ...,
          -75.15764369,  -73.82672994,  -69.96637825],
...
        [-152.78054877, -151.54506003, -150.04756935, ...,
         -154.77175917, -154.31110207, -153.65192904],
        [-138.0274836 , -137.45968028, -136.79212516, ...,
         -139.38518572, -138.96457202, -138.53292622],
        [-144.74204738, -144.90025643, -145.07784789, ...,
         -144.1639217 , -144.3013401 , -144.43820306]],

       [[  29.7472087 ,   29.20563718,   30.22907395, ...,
           28.33649655,   28.10923541,   29.10951473],
        [  27.99637321,   28.92745614,   30.03724924, ...,
           30.72251645,   29.36718763,   28.18875617],
        [  28.68889383,   31.06696563,   32.74818461, ...,
           28.03432011,   28.14676416,   28.59026366],
        ...,
        [ 315.52078138,  315.91532585,  316.17089081, ...,
          312.99636984,  314.06725298,  314.88671616],
        [ 355.21456629,  356.4895354 ,  357.77691957, ...,
          350.91726003,  352.3815619 ,  353.82645206],
        [ 376.43805579,  377.23367521,  378.02273818, ...,
          373.28808444,  374.06752784,  374.86564657]]])
Coordinates:
  * lon      (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * lat      (lat) float64 1.0 3.0 5.0 7.0 9.0 11.0 ... 81.0 83.0 85.0 87.0 89.0
Dimensions without coordinates: run

In [32]:
# Calculate the DJF min 5 trend mean
DJF_min5_trend_mean = DJF_min5_trend.mean(dim='run')
DJF_max5_trend_mean = DJF_max5_trend.mean(dim='run')

# Calculate the JJA min 5 trend mean
JJA_min5_trend_mean = JJA_min5_trend.mean(dim='run')
JJA_max5_trend_mean = JJA_max5_trend.mean(dim='run')

In [40]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm, ListedColormap

# plt.style.use('ggplot')
levels = np.arange(-1000.0, 1050.0, 100.0)
cmap = ListedColormap(sns.color_palette("RdBu_r", n_colors=len(levels)-1).as_hex())

fig = plt.figure(figsize=(15, 8), dpi=300)
gs = gridspec.GridSpec(nrows=2, ncols=2, height_ratios=[1, 1], width_ratios=[1, 1])

ax1 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax1.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax1.coastlines()
ax1.add_feature(cfeature.LAND, facecolor='lightgrey')
ax1.add_feature(cfeature.OCEAN, facecolor='white')

ax1.set_title('a', loc='left', fontsize=12, fontweight='medium')
ax1.set_title('ERA DJF', loc='right', fontsize=12, fontweight='medium')

ax1.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax1.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax1.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax1.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax1.contourf(slp_trend['longitude'], slp_trend['latitude'], slp_trend['DJF'], levels=levels, cmap=cmap, transform=ccrs.PlateCarree())
# cb = plt.colorbar(cf, ax=ax1, orientation='horizontal', pad=0.05, shrink=0.8, extend='both')
# cb.set_label('°C/65yrs', fontsize=10, fontweight='medium')
# cb.ax.tick_params(labelsize=10)

ax2 = fig.add_subplot(gs[0, 1], projection=ccrs.PlateCarree(central_longitude=180))
ax2.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax2.coastlines()
ax2.add_feature(cfeature.LAND, facecolor='lightgrey')
ax2.add_feature(cfeature.OCEAN, facecolor='white')

ax2.set_title('b', loc='left', fontsize=12, fontweight='medium')
ax2.set_title('MPI-ESM-LR DJF MME', loc='right', fontsize=12, fontweight='medium')

ax2.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax2.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax2.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax2.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax2.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax2.contourf(slope_DJF_data['lon'], slope_DJF_data['lat'], slope_DJF_MME, levels=levels, cmap=cmap, transform=ccrs.PlateCarree())
# cb = plt.colorbar(cf, ax=ax2, orientation='horizontal', pad=0.05, shrink=0.8, extend='both')
# cb.set_label('°C/65yrs', fontsize=10, fontweight='medium')
# cb.ax.tick_params(labelsize=10)

ax3 = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax3.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax3.coastlines()
ax3.add_feature(cfeature.LAND, facecolor='lightgrey')
ax3.add_feature(cfeature.OCEAN, facecolor='white')

ax3.set_title('c', loc='left', fontsize=12, fontweight='medium')
ax3.set_title('MPI-ESM-LR DJF Min5-MME', loc='right', fontsize=12, fontweight='medium')

ax3.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax3.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax3.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax3.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax3.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax3.contourf(slope_DJF_data['lon'], slope_DJF_data['lat'], DJF_min5_trend_mean , levels=levels, cmap=cmap, transform=ccrs.PlateCarree())

ax4 = fig.add_subplot(gs[1, 1], projection=ccrs.PlateCarree(central_longitude=180))
ax4.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax4.coastlines()
ax4.add_feature(cfeature.LAND, facecolor='lightgrey')
ax4.add_feature(cfeature.OCEAN, facecolor='white')

ax4.set_title('d', loc='left', fontsize=12, fontweight='medium')
ax4.set_title('MPI-ESM-LR DJF Max5-MME', loc='right', fontsize=12, fontweight='medium')

ax4.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax4.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax4.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax4.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax4.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax4.contourf(slope_DJF_data['lon'], slope_DJF_data['lat'], DJF_max5_trend_mean, levels=levels, cmap=cmap, transform=ccrs.PlateCarree())

plt.subplots_adjust(wspace=0.1, hspace=0.01, top=0.75, bottom=0.05)

cb = plt.colorbar(cf, ax=[ax1, ax2, ax3, ax4], orientation='horizontal', pad=0.05, shrink=0.5, extend='both')
cb.set_label('Pa/15yrs', fontsize=10, fontweight='medium')
cb.ax.tick_params(labelsize=10)

plt.savefig('Obs-MPI-ESM-LR_2008-2022_DJF_SLP_trend_MME.png', bbox_inches='tight', dpi=300)

plt.close()


/tmp/ipykernel_1826017/4249773586.py:102: MatplotlibDeprecationWarning: The 'extend' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(cf, ax=[ax1, ax2, ax3, ax4], orientation='horizontal', pad=0.05, shrink=0.5, extend='both')


In [34]:
fig = plt.figure(figsize=(15, 8), dpi=300)
gs = gridspec.GridSpec(nrows=2, ncols=2, height_ratios=[1, 1], width_ratios=[1, 1])

# Define the levels and colormap
levels = np.arange(-300.0, 325.0, 25)
cmap = ListedColormap(sns.color_palette("RdBu_r", n_colors=len(levels)-1).as_hex())

sig_levels = np.array([-0.05, 0.05])
sig_cmap = ListedColormap(['none', 'none'])

ax1 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax1.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax1.coastlines()
ax1.add_feature(cfeature.LAND, facecolor='lightgrey')
ax1.add_feature(cfeature.OCEAN, facecolor='white')

ax1.set_title('a', loc='left', fontsize=12, fontweight='medium')
ax1.set_title('ERA JJA', loc='right', fontsize=12, fontweight='medium')

ax1.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax1.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax1.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax1.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax1.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax1.contourf(slp_trend['longitude'], slp_trend['latitude'], slp_trend['JJA'], levels=levels, cmap=cmap, extend ='both', transform=ccrs.PlateCarree())
# cb = plt.colorbar(cf, ax=ax1, orientation='horizontal', pad=0.05, shrink=0.8, extend='both')
# cb.set_label('°C/65yrs', fontsize=10, fontweight='medium')
# cb.ax.tick_params(labelsize=10)

ax2 = fig.add_subplot(gs[0, 1], projection=ccrs.PlateCarree(central_longitude=180))
ax2.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax2.coastlines()
ax2.add_feature(cfeature.LAND, facecolor='lightgrey')
ax2.add_feature(cfeature.OCEAN, facecolor='white')

ax2.set_title('b', loc='left', fontsize=12, fontweight='medium')
ax2.set_title('MPI-ESM-LR JJA MME', loc='right', fontsize=12, fontweight='medium')

ax2.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax2.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax2.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax2.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax2.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax2.contourf(slope_JJA_data['lon'], slope_JJA_data['lat'], slope_JJA_MME, levels=levels, cmap=cmap, transform=ccrs.PlateCarree())
# cb = plt.colorbar(cf, ax=ax2, orientation='horizontal', pad=0.05, shrink=0.8, extend='both')
# cb.set_label('°C/65yrs', fontsize=10, fontweight='medium')
# cb.ax.tick_params(labelsize=10)

ax3 = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax3.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax3.coastlines()
ax3.add_feature(cfeature.LAND, facecolor='lightgrey')
ax3.add_feature(cfeature.OCEAN, facecolor='white')

ax3.set_title('c', loc='left', fontsize=12, fontweight='medium')
ax3.set_title('MPI-ESM-LR JJA Min5-MME', loc='right', fontsize=12, fontweight='medium')

ax3.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax3.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax3.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax3.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax3.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax3.contourf(slope_JJA_data['lon'], slope_JJA_data['lat'], JJA_min5_trend_mean , levels=levels, cmap=cmap, transform=ccrs.PlateCarree())

ax4 = fig.add_subplot(gs[1, 1], projection=ccrs.PlateCarree(central_longitude=180))
ax4.set_extent([-180, 180, 0, 90], crs=ccrs.PlateCarree())
ax4.coastlines()
ax4.add_feature(cfeature.LAND, facecolor='lightgrey')
ax4.add_feature(cfeature.OCEAN, facecolor='white')

ax4.set_title('d', loc='left', fontsize=12, fontweight='medium')
ax4.set_title('MPI-ESM-LR JJA Max5-MME', loc='right', fontsize=12, fontweight='medium')

ax4.set_xticks([-180, -120, -60, 0, 60, 120, 180], crs=ccrs.PlateCarree())
ax4.set_yticks([0, 30, 60, 90], crs=ccrs.PlateCarree())
ax4.xaxis.set_major_formatter(cticker.LongitudeFormatter())
ax4.yaxis.set_major_formatter(cticker.LatitudeFormatter())

ax4.tick_params(axis='both', which='major', labelsize=10, direction='out', pad=5)

cf = ax4.contourf(slope_JJA_data['lon'], slope_JJA_data['lat'], JJA_max5_trend_mean, levels=levels, cmap=cmap, transform=ccrs.PlateCarree())

plt.subplots_adjust(wspace=0.1, hspace=0.01, top=0.75, bottom=0.05)

cb = plt.colorbar(cf, ax=[ax1, ax2, ax3, ax4], orientation='horizontal', pad=0.05, shrink=0.5, extend='both')
cb.set_label('Pa/15yrs', fontsize=10, fontweight='medium')
cb.ax.tick_params(labelsize=10)

plt.savefig('Obs-MPI-ESM-LR_2008-2022_JJA_SLP_trend_MME.png', bbox_inches='tight', dpi=300)

plt.close()

/tmp/ipykernel_1826017/751207725.py:91: MatplotlibDeprecationWarning: The 'extend' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.
  cb = plt.colorbar(cf, ax=[ax1, ax2, ax3, ax4], orientation='horizontal', pad=0.05, shrink=0.5, extend='both')
